# Assignment 1:  Sentiment with Deep Neural Networks

Welcome to the first assignment of course 3. In this assignment, you will explore sentiment analysis using deep neural networks. 

In course 1, you implemented Logistic regression and Naive Bayes for sentiment analysis. However if you were to give your old models an example like:

<center> <span style='color:blue'> <b>This movie was almost good.</b> </span> </center>

Your model would have predicted a positive sentiment for that review. However, that sentence has a negative sentiment and indicates that the movie was not good. To solve those kinds of misclassifications, you will write a program that uses deep neural networks to identify sentiment in text. By completing this assignment, you will: 

- Understand how you can build/design a model using layers
- Train a model using a training loop
- Use a binary cross-entropy loss function
- Compute the accuracy of your model
- Predict using your own input

As you can tell, this model follows a similar structure to the one you previously implemented in the second course of this specialization. 
- Indeed most of the deep nets you will be implementing will have a similar structure. The only thing that changes is the model architecture, the inputs, and the outputs. Before starting the assignment, we will introduce you to the Google library `trax` that we use for building and training models.


Now we will show you how to compute the gradient of a certain function `f` by just using `  .grad(f)`. 

- Trax source code can be found on Github: [Trax](https://github.com/google/trax)
- The Trax code also uses the JAX library: [JAX](https://jax.readthedocs.io/en/latest/index.html)


# Part 1:  Import libraries and try out Trax

- Let's import libraries and look at an example of using the Trax library.

In [1]:
import os
import random as rnd

import numpy as np
import trax
from   trax import layers as t
from   trax.fastmath import grad
import trax.fastmath.numpy as fnp

from utils import get_all_tweets, Layer, load_tweets, process_tweet

#trax.supervised.trainer_lib.init_random_number_generators(31)

In [2]:
a = fnp.array(5.)
display(a)
type(a)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


DeviceArray(5., dtype=float32)

jax.interpreters.xla.DeviceArray

In [3]:
def f(x):
    return x**2

In [4]:
print(f'f(a) for a={a}: {f(a)}')

f(a) for a=5.0: 25.0


The gradient (derivative) of function `f` with respect to its input `x` is the derivative of $x^2$.
- The derivative of $x^2$ is $2x$.  
- When x is 5, then $2x=10$.

You can calculate the gradient of a function by using `trax.fastmath.grad(fun=)` and passing in the name of the function.
- In this case the function you want to take the gradient of is `f`.
- The object returned (saved in `grad_f` in this example) is a function that can calculate the gradient of f for a given trax.fastmath.numpy array.

In [5]:
grad_f = grad(fun=f)
type(grad_f)

function

In [6]:
grad_calculation = grad_f(a) # f = x^2 -> d/dx = 2x
display(grad_calculation)

DeviceArray(10., dtype=float32)

# Part 2:  Importing the data

## 2.1  Loading in the data

Import the data set.  
- You may recognize this from earlier assignments in the specialization.
- Details of process_tweet function are available in utils.py file

In [7]:
DATA = '../../../../data/twitter_samples'

In [8]:
all_positive_tweets = get_all_tweets(DATA, 'positive')
all_negative_tweets = get_all_tweets(DATA, 'negative')

In [9]:
len(all_positive_tweets), len(all_negative_tweets)

(5000, 5000)

In [10]:
N_TRAIN = 4000
train_pos = all_positive_tweets[:N_TRAIN]
val_pos = all_positive_tweets[N_TRAIN:]
train_neg = all_negative_tweets[:N_TRAIN]
val_neg = all_negative_tweets[N_TRAIN:]

train_x = train_pos + train_neg
val_x = val_pos + val_neg
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
val_y = np.append(np.ones(len(val_pos)), np.zeros(len(val_neg)))

len(train_x), len(val_x)

(8000, 2000)

In [11]:
print('original:', train_pos[0])
print('processed:', process_tweet(train_pos[0]))

original: #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)
processed: ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


## 2.2  Building the vocabulary

Now build the vocabulary.
- Map each word in each tweet to an integer (an "index"). 
- The following code does this for you, but please read it and understand what it's doing.
- Note that you will build the vocabulary based on the training data. 
- To do so, you will assign an index to everyword by iterating over your training set.

The vocabulary will also include some special tokens
- `__PAD__`: padding
- `</e>`: end of line
- `__UNK__`: a token representing any word that is not in the vocabulary.

In [12]:
vocab = {'__PAD__': 0, '__</e>__': 1, '__UNK__': 2}

i = 3
for tweet in train_x:
    processed = process_tweet(tweet)
    for word in processed:
        if word not in vocab:
            vocab[word] = i
            i += 1
            
len(vocab)            

9111

In [13]:
#display(vocab)

## 2.3  Converting a tweet to a tensor

Write a function that will convert each tweet to a tensor (a list of unique integer IDs representing the processed tweet).
- Note, the returned data type will be a **regular Python `list()`**
    - You won't use TensorFlow in this function
    - You also won't use a numpy array
    - You also won't use trax.fastmath.numpy array
- For words in the tweet that are not in the vocabulary, set them to the unique ID for the token `__UNK__`.

##### Example
Input a tweet:
```CPP
'@happypuppy, is Maria happy?'
```

The tweet_to_tensor will first conver the tweet into a list of tokens (including only relevant words)
```CPP
['maria', 'happi']
```

Then it will convert each word into its unique integer

```CPP
[2, 56]
```
- Notice that the word "maria" is not in the vocabulary, so it is assigned the unique integer associated with the `__UNK__` token, because it is considered "unknown."

### Exercise 01
**Instructions:** Write a program `tweet_to_tensor` that takes in a tweet and converts it to an array of numbers. You can use the `Vocab` dictionary you just found to help create the tensor. 

- Use the vocab_dict parameter and not a global variable.
- Do not hard code the integer value for the `__UNK__` token.

In [14]:
def tweet_to_tensor(tweet, vocab_dict, unk_token='__UNK__', verbose=False):
    '''
    Input: 
        tweet - A string containing a tweet
        vocab_dict - The words dictionary
        unk_token - The special string for unknown tokens
        verbose - Print info durign runtime
    Output:
        tensor_l - A python list with
        
    '''  
    word_l = process_tweet(tweet)
    if verbose:
        print('List of words from the processed tweet:')
        print(word_l)
    tensor_l = []
    unk_ID = vocab_dict[unk_token]
    if verbose:
        print(f'The unique integer ID for the unk_token is {unk_ID}')
    tensor_l = [vocab_dict.get(word, unk_ID) for word in word_l]    
    return tensor_l

In [15]:
print('Actual tweet is\n', val_pos[0])
print(
    '\nTensor of tweet:\n', tweet_to_tensor(val_pos[0], vocab_dict=vocab))

Actual tweet is
 Bro:U wan cut hair anot,ur hair long Liao bo
Me:since ord liao,take it easy lor treat as save $ leave it longer :)
Bro:LOL Sibei xialan

Tensor of tweet:
 [1076, 138, 486, 2365, 755, 8170, 1134, 755, 54, 2, 2688, 801, 2, 2, 355, 609, 2, 3507, 1028, 605, 4579, 9, 1076, 159, 2, 2]


## 2.4  Creating a batch generator

Most of the time in Natural Language Processing, and AI in general we use batches when training our data sets. 
- If instead of training with batches of examples, you were to train a model with one example at a time, it would take a very long time to train the model. 
- You will now build a data generator that takes in the positive/negative tweets and returns a batch of training examples. It returns the model inputs, the targets (positive or negative labels) and the weight for each target (ex: this allows us to can treat some examples as more important to get right than others, but commonly this will all be 1.0). 

Once you create the generator, you could include it in a for loop

```CPP
for batch_inputs, batch_targets, batch_example_weights in data_generator:
    ...
```

You can also get a single batch like this:

```CPP
batch_inputs, batch_targets, batch_example_weights = next(data_generator)
```
The generator returns the next batch each time it's called. 
- This generator returns the data in a format (tensors) that you could directly use in your model.
- It returns a triple: the inputs, targets, and loss weights:
-- Inputs is a tensor that contains the batch of tweets we put into the model.
-- Targets is the corresponding batch of labels that we train to generate.
-- Loss weights here are just 1s with same shape as targets. Next week, you will use it to mask input padding.

### Exercise 02
Implement `data_generator`.

In [24]:
def data_generator(
        data_pos, data_neg, batch_size, loop, vocab_dict, shuffle=False):
    '''
    Input: 
      data_pos - Set of posstive examples
      data_neg - Set of negative examples
      batch_size - number of samples per batch. Must be even
      loop - True or False
      vocab_dict - The words dictionary
      shuffle - Shuffle the data order
    Yield:
      inputs - Subset of positive and negative examples
      targets - The corresponding labels for the subset
      example_weights - An array specifying the importance of each example  
    '''     
    assert batch_size % 2 == 0
    n_to_take = batch_size // 2
    pos_index = 0
    neg_index = 0
    len_data_pos = len(data_pos)
    len_data_neg = len(data_neg)
    pos_index_lines = list(range(len_data_pos))
    neg_index_lines = list(range(len_data_neg))
    if shuffle:
        rnd.shuffle(pos_index_lines)
        rnd.shuffle(neg_index_lines)        
    stop = False
    while not stop:  
        batch = []
        for i in range(n_to_take):
            if pos_index >= len_data_pos: 
                if not loop:
                    stop = True
                    break              
                pos_index = 0
                if shuffle:
                    rnd.shuffle(pos_index_lines)
            tweet = data_pos[pos_index_lines[pos_index]]
            tensor = tweet_to_tensor(tweet, vocab_dict)
            batch.append(tensor)
            pos_index += 1
        for i in range(n_to_take):            
            if neg_index >= len_data_neg:
                if not loop:
                    stop = True
                    break
                neg_index = 0
                if shuffle:
                    rnd.shuffle(neg_index_lines)
            tweet = data_neg[neg_index_lines[neg_index]]
            tensor = tweet_to_tensor(tweet, vocab_dict)
            batch.append(tensor)
            neg_index += 1
        if stop:
            break
        pos_index += n_to_take
        neg_index += n_to_take
        max_len = max([len(t) for t in batch]) 
        tensor_pad_l = []
        for tensor in batch:
            n_pad = max_len - len(tensor)
            pad_l = [0] * n_pad
            tensor_pad = tensor + pad_l
            tensor_pad_l.append(tensor_pad)
        inputs = np.array(tensor_pad_l)
        targets = np.array([1] * n_to_take + [0] * n_to_take)
        example_weights = np.ones_like(targets)
        yield inputs, targets, example_weights

In [25]:
# Set the random number generator for the shuffle procedure
rnd.seed(30) 

# Create the training data generator
def train_generator(batch_size, shuffle=False):
    return data_generator(
        train_pos, train_neg, batch_size, True, vocab, shuffle)

# Create the validation data generator
def val_generator(batch_size, shuffle = False):
    return data_generator(
        val_pos, val_neg, batch_size, True, vocab, shuffle)

# Create the validation data generator
def test_generator(batch_size, shuffle = False):
    return data_generator(
        val_pos, val_neg, batch_size, False, vocab, shuffle)

# Get a batch from the train_generator and inspect.
inputs, targets, example_weights = next(train_generator(4, shuffle=True))

# this will print a list of 4 tensors padded with zeros
print(f'Inputs: {inputs}')
print(f'Targets: {targets}')
print(f'Example Weights: {example_weights}')

Inputs: [[ 349 2019 4471 3218    9    0    0    0    0    0    0]
 [4974  575 2014 1467 5194 3517  143 3517  132  466    9]
 [3780  111  138  591 2946 3989    0    0    0    0    0]
 [ 253 3780    0    0    0    0    0    0    0    0    0]]
Targets: [1 1 0 0]
Example Weights: [1 1 1 1]


In [26]:
# Test the train_generator
# Create a data generator for training data,
# which produces batches of size 4 (for tensors and their respective 
# targets)
tmp_data_gen = train_generator(batch_size = 4)

# Call the data generator to get one batch and its targets
tmp_inputs, tmp_targets, tmp_example_weights = next(tmp_data_gen)

print(f'The inputs shape is {tmp_inputs.shape}')
print(f'The targets shape is {tmp_targets.shape}')
print(f'The example weights shape is {tmp_example_weights.shape}')

for i,t in enumerate(tmp_inputs):
    print(f'input tensor: {t}; target {tmp_targets[i]}; '
          f'example weights {tmp_example_weights[i]}')

The inputs shape is (4, 14)
The targets shape is (4,)
The example weights shape is (4,)
input tensor: [3 4 5 6 7 8 9 0 0 0 0 0 0 0]; target 1; example weights 1
input tensor: [10 11 12 13 14 15 16 17 18 19 20  9 21 22]; target 1; example weights 1
input tensor: [5758 2917 3780    0    0    0    0    0    0    0    0    0    0    0]; target 0; example weights 1
input tensor: [ 868  259 3670 5759  311 4478  575 1241 2783  333 1213 3780    0    0]; target 0; example weights 1


Now that you have your train/val generators, you can just call them and they will return tensors which correspond to your tweets in the first column and their corresponding labels in the second column. Now you can go ahead and start building your neural network. 

# Part 3:  Defining classes

In this part, you will write your own library of layers. It will be very similar
to the one used in Trax and also in Keras and PyTorch. Writing your own small
framework will help you understand how they all work and use them effectively
in the future.

Your framework will be based on the following `Layer` class from utils.py.

```CPP
class Layer(object):
    """ Base class for layers.
    """
      
    # Constructor
    def __init__(self):
        # set weights to None
        self.weights = None

    # The forward propagation should be implemented
    # by subclasses of this Layer class
    def forward(self, x):
        raise NotImplementedError

    # This function initializes the weights
    # based on the input signature and random key,
    # should be implemented by subclasses of this Layer class
    def init_weights_and_state(self, input_signature, random_key):
        pass

    # This initializes and returns the weights, do not override.
    def init(self, input_signature, random_key):
        self.init_weights_and_state(input_signature, random_key)
        return self.weights
 
    # __call__ allows an object of this class
    # to be called like it's a function.
    def __call__(self, x):
        # When this layer object is called, 
        # it calls its forward propagation function
        return self.forward(x)
```

## 3.1  ReLU class
You will now implement the ReLU activation function in a class below.
$$ \mathrm{ReLU}(x) = \mathrm{max}(0,x) $$

### Exercise 03
**Instructions:** Implement the ReLU activation function below. Your function should take in a matrix or vector and it should transform all the negative numbers into 0 while keeping all the positive numbers intact. 

In [ ]:
#------CONTINUE------------#
class Relu(Layer):
    """Relu activation function implementation"""
    def forward(self, x):
        '''
        Input: 
            - x (a numpy array): the input
        Output:
            - activation (numpy array): all positive or 0 version of x
        '''
        ### START CODE HERE (Replace instances of 'None' with your code) ###
        
        activation = None

        ### END CODE HERE ###
        
        return activation

In [ ]:
# Test your relu function
x = np.array([[-2.0, -1.0, 0.0], [0.0, 1.0, 2.0]], dtype=float)
relu_layer = Relu()
print("Test data is:")
print(x)
print("Output of Relu is:")
print(relu_layer(x))